In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
import numpy as np

In [2]:
sc    = SparkContext("local[*]", "MediaVarianza_2")
datos = sc.textFile("../../../datos/data_ok.csv")

In [3]:
num_col = len(datos.collect()[0].split(" "))
if (num_col != 211):
    print("ERROR EN EL CÁLCULO DE COLUMNAS")